## **MIDI Factory**

#### Import Libraries

In [1]:
import xlwings as xw
from midiArabia import MidiArabia

### Constant 

In [2]:
path_midi = "./midi/raw.mid"

In [3]:
song = MidiArabia(path_midi= path_midi,detect_rhythms=False)
dataFrame = song.midi_to_dataFrame()

### Update Excel File

In [4]:
# Connect to the workbook
workbook = xw.Book('test.xlsx')

# Select the worksheet
worksheet = workbook.sheets["sheet1"]

# Delete the existing table (if it exists)
if worksheet.tables:
    worksheet.range(worksheet.tables[0].range.address).api.Delete()

# Add a new table to the worksheet
table = worksheet.tables.add(source=worksheet.range('A1'), name='MyTable')

# Update the table with the new data
table.update(dataFrame)

# Show the updated range of cells in a new window
table.range.api.Select()

True